In [5]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
print(os.listdir("data"))
import tensorflow as tf
import cv2
import seaborn as sns
import matplotlib.pyplot as plt
from PIL import Image

['sample_submission.csv', 'test', 'train', 'train.csv']


In [6]:
print(tf.__version__)

1.15.0


In [7]:
sns.set(font_scale=1.5)

In [12]:
MAIN_DIR = 'data'
TRAIN_DIR = os.path.join(MAIN_DIR,"/train/")
TEST_DIR = os.path.join(MAIN_DIR, "/test/")
IMAGE_ROWS = 32
IMAGE_COLS = 32
CHANNELS = 3
BATCH_SIZE = 256
EPOCHS = 300
IMG_SHAPE = (IMAGE_ROWS, IMAGE_COLS,CHANNELS)

In [14]:
df_train = pd.read_csv(MAIN_DIR + "/train.csv")

In [15]:
df_train['has_cactus'].value_counts()

1    13136
0     4364
Name: has_cactus, dtype: int64

In [16]:
df_train.isnull().any()

id            False
has_cactus    False
dtype: bool

In [17]:
def cnn_graph():
    main_input = tf.keras.Input(shape=(IMAGE_ROWS,IMAGE_COLS,CHANNELS), name="main_input")
    x = tf.keras.layers.Conv2D(16, (3,3), padding='same', use_bias=False, name="conv_1")(main_input)
    x = tf.keras.layers.BatchNormalization(axis=-1)(x)
    x = tf.keras.layers.Activation('relu')(x)

    x = tf.keras.layers.Conv2D(32, (3,3), padding='same', use_bias=False, name='conv_2')(x)
    x = tf.keras.layers.BatchNormalization(axis=-1)(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.Dropout(0.25)(x)

    x = tf.keras.layers.Conv2D(64, (3,3), padding='same', use_bias=False, name="conv_3")(x)
    x = tf.keras.layers.BatchNormalization(axis=-1)(x)
    x = tf.keras.layers.Activation('relu')(x)

    x = tf.keras.layers.Conv2D(64, (3,3), padding='same', use_bias=False, name="conv_4")(x)
    x = tf.keras.layers.BatchNormalization(axis=-1)(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.Dropout(0.25)(x)

    x = tf.keras.layers.Conv2D(128, (3,3), padding='same', use_bias=False, name="conv_5")(x)
    x = tf.keras.layers.BatchNormalization(axis=-1)(x)
    x = tf.keras.layers.Activation('relu')(x)

    x = tf.keras.layers.Conv2D(128, (3,3), padding='same', use_bias=False, name="conv_6")(x)
    x = tf.keras.layers.BatchNormalization(axis=-1)(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.Dropout(0.25)(x)

    x = tf.keras.layers.Conv2D(256, (3,3), padding='same', use_bias=False, name="conv_7")(x)
    x = tf.keras.layers.BatchNormalization(axis=-1)(x)
    x = tf.keras.layers.Activation('relu')(x)

    x = tf.keras.layers.Conv2D(256, (3,3), padding='same', use_bias=False, name="conv_8")(x)
    x = tf.keras.layers.BatchNormalization(axis=-1)(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.Dropout(0.25)(x)

    x = tf.keras.layers.Conv2D(512, (3,3), padding='same', use_bias=False, name="conv_9")(x)
    x = tf.keras.layers.BatchNormalization(axis=-1)(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.Conv2D(512, (3,3), padding='same', use_bias=False, name='conv_10')(x)
    x = tf.keras.layers.BatchNormalization(axis=-1)(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.Dropout(0.25)(x)

    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(256, activation='relu', use_bias=True)(x)
    x = tf.keras.layers.BatchNormalization(axis=-1)(x)
    x = tf.keras.layers.Dropout(0.25)(x)
    outputs = tf.keras.layers.Dense(1, activation='sigmoid', name="output_sigmoid")(x)
    
    model = tf.keras.Model(inputs=main_input, outputs=outputs)
    
    return model

In [19]:
df = pd.read_csv(MAIN_DIR+'/train.csv')
df['has_cactus'] = df['has_cactus'].apply(str)

In [20]:
model = cnn_graph()
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.0001),
                                   loss = 'binary_crossentropy',
                                   metrics=['accuracy'])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [21]:
def load_data(dataframe=None, batch_size=16, class_mode='binary', target_size=(IMAGE_ROWS, IMAGE_COLS)):
    k_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255,
                                                            zoom_range=0.1,
                                                           validation_split=0.1,
                                                           horizontal_flip=True,
                                                           vertical_flip=True,
                                                           fill_mode='nearest')
    data_train = k_gen.flow_from_dataframe(dataframe,
                                           directory=TRAIN_DIR,
                                           x_col='id',
                                           y_col='has_cactus',
                                           has_ext=True,
                                           target_size=target_size,
                                           class_mode=class_mode,
                                           batch_size=batch_size,
                                           shuffle=True,
                                           subset="training")
    
    data_test = k_gen.flow_from_dataframe(dataframe,
                                          directory=TRAIN_DIR,
                                          x_col='id',
                                          y_col='has_cactus',
                                          has_ext=True,
                                          target_size=(target_size),
                                          class_mode=class_mode,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          subset='validation')
    
    return data_train, data_test

In [22]:
train_k_gen, val_k_gen = load_data(dataframe=df,batch_size=BATCH_SIZE, target_size=(IMAGE_ROWS, IMAGE_COLS))

C:\Users\user\anaconda3\envs\tfgpu\lib\site-packages\keras_preprocessing\image\dataframe_iterator.py:282: UserWarning: Found 17500 invalid image filename(s) in x_col="id". These filename(s) will be ignored.
  .format(n_invalid, x_col)


Found 0 validated image filenames belonging to 0 classes.
Found 0 validated image filenames belonging to 0 classes.


In [24]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
main_input (InputLayer)      [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv_1 (Conv2D)              (None, 32, 32, 16)        432       
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 16)        64        
_________________________________________________________________
activation (Activation)      (None, 32, 32, 16)        0         
_________________________________________________________________
conv_2 (Conv2D)              (None, 32, 32, 32)        4608      
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0     

In [25]:
history = model.fit_generator(train_k_gen, 
                              steps_per_epoch=(train_k_gen.n//train_k_gen.batch_size),
                              epochs=EPOCHS,
                              validation_data=val_k_gen,
                              validation_steps=len(val_k_gen),
                              shuffle=True)

Epoch 1/300


ValueError: Empty training data.